In [ ]:
%pip install pandas nltk


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\reaga\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('./final_data_v1.csv')


# Define the labels to remove
labels_to_remove = [
    'track_order', 'edit_account', 'track_refund', 'delivery_options', 'cancel_order',
    'set_up_shipping_address', 'recover_password', 'contact_customer_service', 'contact_human_agent',
    'check_refund_policy', 'change_shipping_address', 'check_cancellation_fee', 'get_refund',
    'check_invoice', 'delete_account', 'get_invoice', 'create_account', 'change_order', 'switch_account'
]

# Remove rows with the specified labels in the 'intent' column
df_cleaned = df[~df['intent'].isin(labels_to_remove)]

# Save the cleaned DataFrame back to a CSV file
df_cleaned.to_csv('drop_intent.csv', index=False)

print(df_cleaned.head())


     flags                                        instruction category  \
4128   BLZ      i try to list your available payment methoids  payment   
4129   BLZ  i nneed help to list the accepted payment methods  payment   
4130  BILP  could you help me check what payment options a...  payment   
4131   BLQ                             see ur payment methods  payment   
4132   BCL  i have got to list the allowed payment modalit...  payment   

                     intent  \
4128  check_payment_methods   
4129  check_payment_methods   
4130  check_payment_methods   
4131  check_payment_methods   
4132  check_payment_methods   

                                               response  has_placeholder  \
4128  Of course! I appreciate your attempt to access...            False   
4129  We really appreciate your inquiry and are more...            False   
4130  We've received your message and requesting ass...            False   
4131  I'll take care of it! I'm here to assist you i...         

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\reaga\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\reaga\AppData\Roaming\nltk_data...


True

In [ ]:
import pandas as pd
import random
from nltk.corpus import wordnet

# Function to get synonyms for a word
def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    return list(synonyms)

# Function to replace words with their synonyms
def synonym_replacement(sentence, n=1):
    words = sentence.split()
    new_words = words.copy()
    random_word_list = list(set([word for word in words if get_synonyms(word)]))
    random.shuffle(random_word_list)
    num_replaced = 0

    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        if len(synonyms) > 0:
            synonym = random.choice(synonyms).replace("_", " ")
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced >= n:
            break

    return " ".join(new_words)

# Function to insert random words into the sentence
def random_word_insertion(sentence, n=1):
    words = sentence.split()
    new_words = words.copy()

    for _ in range(n):
        random_word = random.choice(words)
        random_idx = random.randint(0, len(new_words))
        new_words.insert(random_idx, random_word)

    return " ".join(new_words)

# Function to delete random words from the sentence
def random_word_deletion(sentence, p=0.1):
    words = sentence.split()
    new_words = [word for word in words if random.random() > p]
    if len(new_words) == 0:  # Ensure we don't return an empty sentence
        return sentence
    return " ".join(new_words)

# Main function to augment data
def augment_data(df, augmentations_per_sentence=3):
    augmented_data = []

    for _, row in df.iterrows():
        instruction = row['instruction']
        intent = row['intent']

        # Apply synonym replacement
        augmented_data.append({
            'instruction': synonym_replacement(instruction),
            'intent': intent
        })

        # Apply random word insertion
        augmented_data.append({
            'instruction': random_word_insertion(instruction),
            'intent': intent
        })

        # Apply random word deletion
        augmented_data.append({
            'instruction': random_word_deletion(instruction),
            'intent': intent
        })

    return pd.DataFrame(augmented_data)

# Load your dataset
csv_file = "./drop_intent.csv"  # Replace with your file path
df = pd.read_csv(csv_file)


# Augment the dataset
augmented_df = augment_data(df)

# Save the augmented dataset to a new CSV file
augmented_df.to_csv("augmented_dataset1.csv", index=False)

print("Data augmentation complete! Augmented dataset saved as 'augmented_dataset.csv'.")


Data augmentation complete! Augmented dataset saved as 'augmented_dataset.csv'.


In [8]:
# Load your dataset
csv_file = "augmented_dataset1.csv"  # Replace with your file path
df = pd.read_csv(csv_file)

print(len(df))


23859
